In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_predict, cross_val_score

In [3]:
data = pd.read_csv(r'/Users/allen/Downloads/HIGGS.csv.gz',compression='gzip',nrows=50000,header=None)

In [7]:
test = pd.read_csv(r'/Users/allen/Downloads/HIGGS.csv.gz',compression='gzip', skiprows=500000 , nrows=50000,header=None)

In [14]:
rf_model = RandomForestClassifier()
lr_model = LogisticRegression()

In [26]:
targets = data[0]
x = data.loc[:,1:]

In [31]:
lr_preds = cross_val_predict(lr_model,x,targets,cv=5,method='predict_proba')

In [32]:
rf_preds = cross_val_predict(rf_model,x,targets,cv=5,method='predict_proba')


In [33]:
lr_preds

array([[0.37355048, 0.62644952],
       [0.38052229, 0.61947771],
       [0.3242798 , 0.6757202 ],
       ...,
       [0.46805482, 0.53194518],
       [0.49427023, 0.50572977],
       [0.40113915, 0.59886085]])

In [34]:
cross_val_score(rf_model,x,targets,cv=5)
cross_val_score(lr_model,x,targets,cv=5)

array([0.6456, 0.6451, 0.6399, 0.6352, 0.6353])

In [35]:
rf_preds[:,1]
lr_preds[:,1]

array([0.62644952, 0.61947771, 0.6757202 , ..., 0.53194518, 0.50572977,
       0.59886085])

In [42]:
new = pd.DataFrame(rf_preds[:,1],columns=['rf'])

In [43]:
new['lr'] = lr_preds[:,1]

In [44]:
new

,rf,lr
0,0.63,0.626450
1,0.84,0.619478
2,0.93,0.675720
3,0.70,0.600039
4,0.62,0.485716
...,...,...
49995,0.60,0.806925
49996,0.28,0.626240
49997,0.42,0.531945
49998,0.66,0.505730


In [47]:
from sklearn.linear_model import LinearRegression
final = LinearRegression()
final_preds = cross_val_predict(final,new,targets,cv=5)

In [48]:
final_preds

array([0.66537048, 0.91222472, 1.02674504, ..., 0.39860522, 0.67878276,
       0.45641223])

In [49]:
guess = pd.DataFrame(final_preds,columns = ['Final_preds'])

In [50]:
guess['predict'] = 0

In [51]:
guess.loc[guess['Final_preds']>0.5,'predict'] = 1

In [52]:
from sklearn.metrics import accuracy_score
accuracy_score(guess['predict'],targets)

0.71358

***
## Blending

In [54]:
y = data[0]
blended = data.iloc[:,-4:]
fl_data = data.iloc[:,1:25]

In [55]:
rf_preds = cross_val_predict(rf_model,fl_data,y,cv=5,method='predict_proba')

In [56]:
lr_preds = cross_val_predict(rf_model,fl_data,y,cv=5,method='predict_proba')

In [57]:
fl_data['lr_preds'] = lr_preds[:,1]

/var/folders/cn/_lgzxbk177j14_b82nbl10br0000gn/T/ipykernel_23653/3251341083.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fl_data['lr_preds'] = lr_preds[:,1]


In [58]:
fl_data['rf_preds'] = rf_preds[:,1]

/var/folders/cn/_lgzxbk177j14_b82nbl10br0000gn/T/ipykernel_23653/438224341.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fl_data['rf_preds'] = rf_preds[:,1]


In [59]:
fl_data

,1,2,3,4,5,6,7,8,9,10,...,17,18,19,20,21,22,23,24,lr_preds,rf_preds
0,0.869293,-0.635082,0.225690,0.327470,-0.689993,0.754202,-0.248573,-1.092064,0.000000,1.374992,...,0.000000,0.657930,-0.010455,-0.045767,3.101961,1.353760,0.979563,0.978076,0.60,0.60
1,0.907542,0.329147,0.359412,1.497970,-0.313010,1.095531,-0.557525,-1.588230,2.173076,0.812581,...,0.000000,0.398701,-1.138930,-0.000819,0.000000,0.302220,0.833048,0.985700,0.56,0.57
2,0.798835,1.470639,-1.635975,0.453773,0.425629,1.104875,1.282322,1.381664,0.000000,0.851737,...,2.548224,1.256955,1.128848,0.900461,0.000000,0.909753,1.108330,0.985692,0.69,0.76
3,1.344385,-0.876626,0.935913,1.992050,0.882454,1.786066,-1.646778,-0.942383,0.000000,2.423265,...,0.000000,0.745313,-0.678379,-1.360356,0.000000,0.946652,1.028704,0.998656,0.53,0.54
4,1.105009,0.321356,1.522401,0.882808,-1.205349,0.681466,-1.070464,-0.921871,0.000000,0.800872,...,0.000000,0.479999,-0.373566,0.113041,0.000000,0.755856,1.361057,0.986610,0.54,0.53
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,0.620218,-0.823058,0.157442,1.442791,0.767518,2.327923,0.344575,-0.072010,0.000000,1.762271,...,0.000000,1.535169,-2.329035,1.499211,0.000000,1.733151,1.019404,0.991311,0.54,0.60
49996,0.686833,-1.711317,-1.417914,1.311205,-0.298519,0.517214,-1.012040,-1.190188,0.000000,0.570343,...,0.000000,1.006489,-1.039824,0.303930,3.101961,0.715210,0.735663,1.112954,0.55,0.41
49997,0.728559,-1.834037,-1.397939,0.344134,-0.390320,0.951158,-0.165394,-1.400851,0.000000,1.260420,...,2.548224,0.595617,2.004978,0.829432,0.000000,0.636411,0.983770,0.996603,0.60,0.56
49998,1.477981,-0.818188,-1.546087,0.835522,-1.267488,0.785623,0.013838,0.079991,2.173076,0.677361,...,2.548224,0.460526,-0.616750,0.572508,0.000000,0.526324,0.893976,0.977501,0.46,0.43


In [60]:
rf_final = RandomForestClassifier()
rf_final.fit(fl_data,y)

/Users/allen/virtualenvs/QTW/lib/python3.9/site-packages/sklearn/utils/validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(


RandomForestClassifier()

***
## Out of Fold Predictions

In [62]:
x_train = data.iloc[:,1:]
y_train = data.iloc[:,0]

In [63]:
lr_preds = cross_val_predict(lr_model,x_train,y_train,cv=5,method='predict_proba')
rf_preds = cross_val_predict(rf_model,x_train,y_train,cv=5,method='predict_proba')

In [64]:
rf_model2 = RandomForestClassifier()
rf_model2.fit(x_train,y_train)

RandomForestClassifier()

In [65]:
lr_model2 = LogisticRegression()
lr_model2.fit(x_train,y_train)

LogisticRegression()

In [66]:
level2_data=pd.DataFrame(rf_preds[:,1],columns=['rf'])
level2_data['lr'] = lr_preds[:,1]

In [68]:
linear_final_oof = LinearRegression()
linear_final_oof.fit(level2_data,y_train)

LinearRegression()

In [69]:
rf_infold = rf_model2.predict_proba(x_train)
lr_infold = lr_model2.predict_proba(x_train)

In [70]:
level2_infold = pd.DataFrame(rf_infold[:,1],columns=['rf'])
level2_infold['lr'] = lr_infold[:,1]

In [72]:

linear_final_if = LinearRegression()
linear_final_if.fit(level2_infold,y_train)

LinearRegression()

In [73]:
x_test = test.iloc[:,1:]
y_test = test.iloc[:,0]

In [74]:
test_rf = rf_model2.predict_proba(x_test)
test_lr = lr_model2.predict_proba(x_test)

In [75]:
test_final = pd.DataFrame(test_rf)

In [76]:
test_final['lr'] = test_lr[:,1]

In [77]:
test_final

,0,1,lr
0,0.50,0.50,0.480288
1,0.52,0.48,0.403893
2,0.49,0.51,0.433524
3,0.26,0.74,0.717046
4,0.41,0.59,0.844457
...,...,...,...
49995,0.39,0.61,0.562244
49996,0.40,0.60,0.648267
49997,0.77,0.23,0.424575
49998,0.50,0.50,0.759679


In [82]:
test_final = test_final.drop(columns=[0])

In [83]:
oof_preds = linear_final_oof.predict(test_final)

/Users/allen/virtualenvs/QTW/lib/python3.9/site-packages/sklearn/utils/validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
/Users/allen/virtualenvs/QTW/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


In [84]:
if_preds = linear_final_if.predict(test_final)

/Users/allen/virtualenvs/QTW/lib/python3.9/site-packages/sklearn/utils/validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
/Users/allen/virtualenvs/QTW/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


In [85]:
from sklearn.metrics import roc_auc_score

In [87]:
roc_auc_score(y_test,oof_preds)

0.7922154135789652

In [88]:
roc_auc_score(y_test,if_preds)

0.7860547616615063

***
## Multi-level Ensembling

## Level 1

In [98]:
rf_level1 = RandomForestClassifier()
lr_level1 = LogisticRegression()

In [99]:
rf1 = cross_val_predict(rf_level1,x_train,y_train,cv=3,method='predict_proba')
lr1 = cross_val_predict(lr_level1,x_train,y_train,cv=3,method='predict_proba')

In [100]:
rf_level2 = RandomForestClassifier()
lr_level2 = LogisticRegression()

In [101]:
level2_data = pd.DataFrame(rf1[:,1:])
level2_data['lr'] = lr1[:,1]

In [102]:
rf2 = cross_val_predict(rf_level2, level2_data,y_train,cv=3,method='predict_proba')
lr2 = cross_val_predict(lr_level2, level2_data,y_train,cv=3,method='predict_proba')

/Users/allen/virtualenvs/QTW/lib/python3.9/site-packages/sklearn/utils/validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
/Users/allen/virtualenvs/QTW/lib/python3.9/site-packages/sklearn/utils/validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
/Users/allen/virtualenvs/QTW/lib/python3.9/site-packages/sklearn/utils/validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
/Users/allen/virtualenvs/QTW/lib/python3.9/site-packages/sklearn/utils/validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be rais

In [103]:
final_model = LinearRegression()

In [104]:
level3_data = pd.DataFrame(rf2[:,1])
level3_data['lr'] = lr2[:,1]

In [105]:
final_model.fit(level3_data,y_train)

/Users/allen/virtualenvs/QTW/lib/python3.9/site-packages/sklearn/utils/validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(


LinearRegression()